In [1]:
import pandas as pd
import ipywidgets as widgets


In [2]:
airports = pd.read_csv("../data/aeroportsANLO.csv")
routes = pd.read_csv("../data/routesANLO.csv")

In [69]:
# creation du list from airports 
name_airports = airports['name'].tolist()
iata_airports = airports['iata'].tolist()

In [67]:
airportDepart = widgets.Dropdown(
    options= name_airports,
    description='Airport Depart:',
    disabled=False,
)
airportDepart

Dropdown(description='Airport Depart:', options=('A Coruna Airport', 'Aachen Merzbruck Airport', 'Aalborg Airp…

In [68]:
airportArrive = widgets.Dropdown(
    options= name_airports,
    description='Airport Depart:',
    disabled=False,
)

airportArrive

Dropdown(description='Airport Depart:', options=('A Coruna Airport', 'Aachen Merzbruck Airport', 'Aalborg Airp…

In [6]:
nb_correspondances =widgets.BoundedIntText(
    value=1,
    min=1,
    max=5,
    step=1,
    description='Number of correpondances:',
    disabled=False
)

nb_correspondances

BoundedIntText(value=1, description='Number of correpondances:', max=5, min=1)

In [7]:
"On veut aller de {} à {} avec {} corresponances".format(airportDepart.value, airportArrive.value, nb_correspondances.value)


'On veut aller de A Coruna Airport à A Coruna Airport avec 1 corresponances'

In [75]:
# d'abord on vais ecrire une function pour trouver une route entre deux airports donnees 
def findAirportCode(airportName : str):
    '''
    fonction pour trouver le airport code ('iata') si on a une airportname 
    retour : string with 3 letters corresponding to iata of airport 
    '''
    index = name_airports.index(airportName)
    return iata_airports[index]
        

In [188]:
# fonction qui vérifie si une les deux codes de airports ont une liaison/route 
def connection_between_two_airports(airportArriveCode : str, airportDepartCode : str):
    '''
    fonction qui verifie si on a une liaison/route entre les deux codes 
    retour : integer qui répresent le num_IATA 
    '''
    if(airportDepartCode == ''):
      tmp1 = ["airportC", "IATA"]
      tmp = dict();
      index = 0 ; 
      for row in routes.iterrows():
        if(row[1][11] == airportArriveCode):
          tmp2 = [row[1][12], row[1][13]]
          tmp3 = dict(zip(tmp1, tmp2))
          tmp[index] = tmp3 
          index = index + 1
      return tmp
    else: 
      for row in routes.iterrows():
        # 11 -> arrive, 12 --> depart, 13 --> num_IATA 
        if((row[1][11] == airportArriveCode) and (row[1][12] == airportDepartCode)):
          return row[1][13]
        


In [189]:
connection_between_two_airports('JFK', '')

{0: {'airportC': 'MST', 'IATA': 'CC607'},
 1: {'airportC': 'SFO', 'IATA': 'AS186'},
 2: {'airportC': 'SFO', 'IATA': 'AS27'},
 3: {'airportC': 'SAN', 'IATA': 'AS178'},
 4: {'airportC': 'SEA', 'IATA': 'AS77'},
 5: {'airportC': 'MXP', 'IATA': 'AZ9407'},
 6: {'airportC': 'DEL', 'IATA': 'AA292'},
 7: {'airportC': 'LAX', 'IATA': 'AA1'},
 8: {'airportC': 'LAX', 'IATA': 'AA171'},
 9: {'airportC': 'PHX', 'IATA': 'AA2369'},
 10: {'airportC': 'MBJ', 'IATA': 'AA682'},
 11: {'airportC': 'PUJ', 'IATA': 'AA1485'},
 12: {'airportC': 'SFO', 'IATA': 'AA76'},
 13: {'airportC': 'STT', 'IATA': 'AA936'},
 14: {'airportC': 'MIA', 'IATA': 'AA1610'},
 15: {'airportC': 'SJD', 'IATA': 'AA2665'},
 16: {'airportC': 'GEO', 'IATA': 'AA2557'},
 17: {'airportC': 'ORD', 'IATA': 'AA2819'},
 18: {'airportC': 'AUS', 'IATA': 'AA2813'},
 19: {'airportC': 'ANU', 'IATA': 'AA1013'},
 20: {'airportC': 'CUN', 'IATA': 'AA1327'},
 21: {'airportC': 'DFW', 'IATA': 'AA2234'},
 22: {'airportC': 'CLT', 'IATA': 'AA495'},
 23: {'airportC

In [ ]:
def liaison_correspondances(nb_correspondances : int, airportName1 : str, airportName2 : str): 
    c1 = findAirportCode(airportName1)
    c2 = findAirportCode(airportName2)
    
    if(nb_correspondances == 1):
        return connection_between_two_airports(c1, c2) 
    else:
        l1 = connection_between_two_airports(c1, '')
        l2 = connection_between_two_airports(c2, '')
        
        
        